In [4]:
import csv
import sqlite3
from collections import defaultdict

from tqdm import tqdm

from src.config import TLCPaths

In [37]:
db_path = "/home/tim/PycharmProjects/py-umls/databases/umls.db"
db = sqlite3.connect(db_path)
cur = db.cursor()


def get_semantic_type(cui):
    sql = f"SELECT * FROM descriptions WHERE CUI = ? LIMIT 1"

    try:
        res = next(cur.execute(sql, (cui,)))[-1]
        semantic_types = res.split("|")
    except StopIteration:
        #print('Semantic type for cui ' + "'" + str(identifier) + "'" + ' not found')
        semantic_types = ["T109"]
    return semantic_types


In [38]:
german_cuis = defaultdict(list)
with open(TLCPaths.project_data_path / "german_umls_names_and_cuis.csv", newline='') as csvfile:
    reader = csv.reader(csvfile, delimiter=' ', quotechar='|')
    for row in reader:
        assert len(row) == 2
        german_cuis[row[0]].append(row[1].strip())

In [39]:
xmen_cuis = []
for cui, names in tqdm(german_cuis.items()):
    s_types = get_semantic_type(cui)
    canonical_name = names[0]
    description = "__definition_missing__"
    aliases = names
    xmen_cuis.append(
        dict(concept_id=cui, canonical_name=canonical_name, definition=description, aliases=aliases,
             types=s_types))

100%|██████████| 109629/109629 [00:01<00:00, 65532.81it/s]


In [40]:
import jsonlines

with jsonlines.open(TLCPaths.project_data_path / "xmen_german_tlc.jsonl", mode='w') as writer:
    writer.write_all(xmen_cuis)